In [ ]:
!nvidia-smi

Wed Jun 28 12:43:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: accelerate 0.20.3
Uninstalling accelerate-0.20.3:
  Successfully uninstalled accelerate-0.20.3
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)


In [ ]:
import pandas as pd
import nltk
import torch
import matplotlib.pyplot as plt

from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
from nltk.tokenize import sent_tokenize
from tqdm import tqdm

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

### Code Explanation

1. **model_ckpt = "google/pegasus-cnn_dailymail"**: This line assigns a
string value "google/pegasus-cnn_dailymail" to the variable model_ckpt. This string represents the pre-trained checkpoint or model name for the PEGASUS model trained on the CNN/Daily Mail dataset.

2. **tokenizer = AutoTokenizer.from_pretrained(model_ckpt)**: This line creates a tokenizer object by calling the from_pretrained method of the AutoTokenizer class. The AutoTokenizer class automatically selects an appropriate tokenizer based on the model checkpoint name (model_ckpt) provided. In this case, it will use the tokenizer specific to the PEGASUS model.

3. **model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)**: This line loads the pre-trained PEGASUS model for sequence-to-sequence language generation. It uses the AutoModelForSeq2SeqLM class and its from_pretrained method to load the model checkpoint specified by model_ckpt. The loaded model is then moved to a specified device (e.g., GPU) using the .to(device) method, which enables utilizing hardware acceleration if available.

In [ ]:
# Downloading the Data
!wget https://github.com/alfandysurya2/project_datasets/raw/main/TextSummarizerDatasets/summarizer-data.zip
!unzip summarizer-data.zip

--2023-06-28 13:15:40--  https://github.com/alfandysurya2/project_datasets/raw/main/TextSummarizerDatasets/summarizer-data.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alfandysurya2/project_datasets/main/TextSummarizerDatasets/summarizer-data.zip [following]
--2023-06-28 13:15:41--  https://raw.githubusercontent.com/alfandysurya2/project_datasets/main/TextSummarizerDatasets/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.04s   

20

In [ ]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"split lengths: {split_lengths}")
print(f"features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum['test'][1]['dialogue'])
print("\nSummary:")

print(dataset_samsum['test'][1]['summary'])

split lengths: [14732, 819, 818]
features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [ ]:
def convert_examples_to_features(data):
  input_encodings = tokenizer(data['dialogue'], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(data['summary'], max_length = 128, truncation=True)

  return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

### Code Explanation

1. **input_encodings = tokenizer(data['dialogue'], max_length=1024, truncation=True)**: This line uses the tokenizer object to tokenize and encode the dialogue data from the data dictionary. It passes the dialogue text as the input to the tokenizer's __call__ method. The max_length parameter specifies the maximum length allowed for the input sequence, and truncation=True indicates that any input sequence longer than the specified maximum length should be truncated. The resulting tokenized and encoded data is stored in the input_encodings variable.

2. **with tokenizer.as_target_tokenizer():**: This line indicates that the subsequent block of code should be executed with the tokenizer acting as a target tokenizer. In some tokenizers, this context manager allows you to specify different behavior for tokenizing and encoding target-specific sequences, such as summaries or labels.

3. **target_encodings = tokenizer(data['summary'], max_length=128, truncation=True)**: Within the context manager, this line tokenizes and encodes the summary data from the data dictionary. It uses the same tokenizer but with target-specific settings. The max_length parameter limits the maximum length of the target sequence, and truncation=True ensures that any sequence longer than the specified maximum length is truncated. The resulting tokenized and encoded data is stored in the target_encodings variable.

4. **The function then returns a dictionary containing the processed data**:
  - **input_ids**: input_encodings['input_ids'] retrieves the token IDs of the input sequence and assigns them to the 'input_ids' key in the returned dictionary.
  - **attention_mask**: input_encodings['attention_mask'] retrieves the attention mask of the input sequence (indicating which tokens are important) and assigns it to the 'attention_mask' key.
  - **labels**: target_encodings['input_ids'] retrieves the token IDs of the target sequence (summary or label) and assigns them to the 'labels' key in the returned dictionary.

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset_samsum_pt['train'][1]

{'id': '13728867',
 'dialogue': 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
 'summary': 'Olivia and Olivier are voting for liberals in this election. ',
 'input_ids': [18038,
  151,
  2632,
  127,
  119,
  6228,
  118,
  115,
  136,
  2974,
  152,
  10463,
  151,
  35884,
  130,
  329,
  107,
  18038,
  151,
  2587,
  314,
  1242,
  10463,
  151,
  1509,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]}